In [11]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [130]:
import pandas as pd
import regex as re
import nltk
import csv
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
import string
import csv
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns


nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')
nltk.download('sentiwordnet')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [131]:
df = pd.read_csv("/kaggle/input/cyberbullydataset/cyber_bullying_dataset.csv")

In [134]:
df.shape

(3592, 2)

In [137]:
# Cek dataset jika ada yang duplikat dan jika ada yang kosong dihapus
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [138]:
df.shape

(3591, 2)

In [133]:
# Display first few rows and dataset info
print(df.head(20))

                                          raw_content  \
0   Sorry walau orang desa ga ga bulluk. ga dekil ...   
1   Adzan buat panggilan kampanye semoga aja nggak...   
2   gw ga sampe bundir sih preventif biar ga stres...   
3                        Cupu https://t.co/tP58k5u17Z   
4   pengen banget sibuk ngurusin keperluan ospek, ...   
5   Aku benci banget sama satu budheku. Setiap kal...   
6   @ub_mfs Biar gaada pembeda, kalian itu di ospe...   
7       @RagilSemar Itu pinokio, boneka goblok dungu,   
8   malam semuanyaa!! ‍️ boleh tolong ospekin setu...   
9   新しい  :  ospEk timE!  ★  hello, kindly ospekin ...   
10               Ngelatih mental kok ospek, nikah lah   
11  Help repost/retweet please? Hai! Boleh bantu o...   
12  help ospek &amp; sarannnya teman-teman https:/...   
13  @tanyakanrl But mamaku sampe detik ini masih p...   
14  NAJISSSSS!!! Badan gw cakep juga bukan buat la...   
15                   @februar_ry Dulu jelek dan kurus   
16  Ternyata, tidak ada hal yan

In [80]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   raw_content  3592 non-null   object
 1   content      3591 non-null   object
dtypes: object(2)
memory usage: 56.2+ KB
None


In [81]:
# Hapus kolom 'raw_content'
df.drop(columns=['raw_content'], inplace=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  3591 non-null   object
dtypes: object(1)
memory usage: 28.2+ KB


In [83]:
df.head(20)

,content
0,Sorry walau orang desa ga ga bulluk. ga dekil ...
1,Adzan buat panggilan kampanye semoga aja nggak...
2,gw ga sampe bundir sih preventif biar ga stres...
3,Cupu
4,"pengen banget sibuk ngurusin keperluan ospek, ..."
5,Aku benci banget sama satu budheku. Setiap kal...
6,"Biar gaada pembeda, kalian itu di ospek sama s..."
7,"Itu pinokio, boneka goblok dungu,"
8,malam semuanyaa!! ‍️ boleh tolong ospekin setu...
9,"新しい : ospEk timE! ★ hello, kindly ospekin anak..."


In [84]:
df.shape

(3592, 1)

In [20]:
# # Function to clean text
# def text_preprocessing(text):
#     text = text.lower()  # Lowercase
#     text = re.sub(r'@\w+', '', text)  # Remove mentions
#     text = re.sub(r'#\w+', '', text)  # Remove tags
#     text = re.sub(r'https?://\S+', '', text)  # Remove URLs
#     text = re.sub(r'[-+]?[0-9]+', '', text)  # Remove numbers
#     text = re.sub(r'[0-9]', '', text)  # Remove numbers
#     text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#     text = text.strip()  # Remove leading and trailing whitespaces
#     text = re.sub(r'\s{2,}', ' ', text)  # Remove extra spaces
#     return text


In [21]:
# # Function to remove repeated words
# def remove_repeated_words(text):
#     pattern = r'\b(\w+)(\s+\1)+\b'
#     return re.sub(pattern, r'\1', text)

# # Function to replace words in sentence based on dictionary
# def replace_words_in_sentence(df, column, dictionary):
#     def replace_word(word):
#         return dictionary.get(word, word)

#     def replace_sentence(sentence):
#         return ' '.join(replace_word(word) for word in sentence.split())

#     df[column] = df[column].apply(replace_sentence)
#     return df

# Data Pre Processing 

In [85]:
# Function Cleaning Text
# Menghapus text yang sekiranya tidak digunakan pada penilitian kali ini seperti mention tags link, dll
def removeUnsualWords(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub(r'\@([\w]+)',' ', text)
    text = re.sub(r'\#([\w]+)',' ', text)
    text = re.sub(r'http\S+', '',text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)
    text = re.sub(r'([a-zA-Z])\1\1','\\1', text)
    text = re.sub(' +', ' ', text)
    text = re.sub(r'^[ ]|[ ]$','', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation)) # Menghapus tanda baca
    text = text.strip(' ')
    return text

# Mengubah semua huruf dalam teks menjadi huruf kecil.

In [86]:
# Lower Text Function
# Mengubah semua alphabet menjadi huruf kecil
def tolowerText(text):
    text = text.lower()
    return text

# Mengubah kalimat menjadi daftar kata (token).

In [87]:
# Tokenize Word
# meng-tokenize kalimat yang ada pada data
def tokenizingText(text):
    text = word_tokenize(text)
    return text


# Mengambil daftar stopword dalam bahasa Indonesia dan Menghapus stopword dari daftar kata yang telah ditokenisasi.

In [88]:
# Filter Text
# Cek jika data masih mengandung kalimat bukan bahasa Indonesia
def filteringText(text):
    listText = set(stopwords.words('indonesian'))
    filteredText = []
    for txt in text :
        if txt not in listText:
            filteredText.append(txt)
    text = filteredText
    return text


# Membuat objek stemmer dari Sastrawi dan Melakukan stemming pada setiap kata dalam teks.

In [89]:
# Steming Function
def stemmingText(text):
    sf = StemmerFactory()
    stemmer = sf.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

# Menggabungkan daftar kata menjadi sebuah kalimat.

In [90]:
# To Sentence Function
# Mengembalikkan kata yang sudah di proses menjadi kalimat kembali
def toSentence(listWords):
    sentence = ' '.join(word for word in listWords)
    return sentence

In [91]:
# Create a copy of the dataframe for normalization and labeling
xx = df.copy()
# Ensure all values in 'content' are strings and handle NaN values
df['content'] = df['content'].astype(str).fillna('')

In [92]:
df['clean_text'] = df['content'].apply(removeUnsualWords)

In [93]:
df['clean_text'] = df['clean_text'].apply(tolowerText)

In [94]:
df['preprocess_text'] = df['clean_text'].apply(tokenizingText)

In [95]:
df['preprocess_text'] = df['preprocess_text'].apply(filteringText)

In [109]:
df.head(40)

,content,clean_text,preprocess_text
0,Sorry walau orang desa ga ga bulluk. ga dekil ...,sorry walau orang desa ga ga bulluk ga dekil k...,"[sorry, orang, desa, ga, ga, bulluk, ga, dekil..."
1,Adzan buat panggilan kampanye semoga aja nggak...,adzan buat panggilan kampanye semoga aja nggak...,"[adzan, panggilan, kampanye, semoga, aja, ngga..."
2,gw ga sampe bundir sih preventif biar ga stres...,gw ga sampe bundir sih preventif biar ga stres...,"[gw, ga, sampe, bundir, sih, preventif, biar, ..."
3,Cupu,cupu,[cupu]
4,"pengen banget sibuk ngurusin keperluan ospek, ...",pengen banget sibuk ngurusin keperluan ospek k...,"[pengen, banget, sibuk, ngurusin, keperluan, o..."
5,Aku benci banget sama satu budheku. Setiap kal...,aku benci banget sama satu budheku setiap kali...,"[benci, banget, budheku, kali, ketemu, gak, mu..."
6,"Biar gaada pembeda, kalian itu di ospek sama s...",biar gaada pembeda kalian itu di ospek sama sa...,"[biar, gaada, pembeda, ospek, mahasiswa, ank, ..."
7,"Itu pinokio, boneka goblok dungu,",itu pinokio boneka goblok dungu,"[pinokio, boneka, goblok, dungu]"
8,malam semuanyaa!! ‍️ boleh tolong ospekin setu...,malam semuanyaa boleh tolong ospekin setup dan...,"[malam, semuanyaa, tolong, ospekin, setup, lay..."
9,"新しい : ospEk timE! ★ hello, kindly ospekin anak...",ospek time hello kindly ospekin anak baru aku ...,"[ospek, time, hello, kindly, ospekin, anak, te..."


In [110]:
df['preprocess_text'] = df['preprocess_text'].apply(stemmingText)

In [111]:
df.head(40)

,content,clean_text,preprocess_text
0,Sorry walau orang desa ga ga bulluk. ga dekil ...,sorry walau orang desa ga ga bulluk ga dekil k...,"[sorry, orang, desa, ga, ga, bulluk, ga, dekil..."
1,Adzan buat panggilan kampanye semoga aja nggak...,adzan buat panggilan kampanye semoga aja nggak...,"[adzan, panggil, kampanye, moga, aja, nggak, b..."
2,gw ga sampe bundir sih preventif biar ga stres...,gw ga sampe bundir sih preventif biar ga stres...,"[gw, ga, sampe, bundir, sih, preventif, biar, ..."
3,Cupu,cupu,[cupu]
4,"pengen banget sibuk ngurusin keperluan ospek, ...",pengen banget sibuk ngurusin keperluan ospek k...,"[ken, banget, sibuk, ngurusin, perlu, ospek, p..."
5,Aku benci banget sama satu budheku. Setiap kal...,aku benci banget sama satu budheku setiap kali...,"[benci, banget, budheku, kali, ketemu, gak, mu..."
6,"Biar gaada pembeda, kalian itu di ospek sama s...",biar gaada pembeda kalian itu di ospek sama sa...,"[biar, gaada, beda, ospek, mahasiswa, ank, gaa..."
7,"Itu pinokio, boneka goblok dungu,",itu pinokio boneka goblok dungu,"[pinokio, boneka, goblok, dungu]"
8,malam semuanyaa!! ‍️ boleh tolong ospekin setu...,malam semuanyaa boleh tolong ospekin setup dan...,"[malam, semuanyaa, tolong, ospekin, setup, lay..."
9,"新しい : ospEk timE! ★ hello, kindly ospekin anak...",ospek time hello kindly ospekin anak baru aku ...,"[ospek, time, hello, kindly, ospekin, anak, in..."


# Tambahan Pre Procesing

In [22]:
# Additional cleaning steps
def additional_text_cleaning(text):
    # Remove URLs
    text = re.sub(r'http\S+\s*','', str(text))
    # Remove Mention, Hashtag, & Special Characters
    text = re.sub(r'@[a-zA-Z_0-9]*','', str(text))  # User Mention
    text = re.sub(r'#[a-zA-Z_0-9]*','', str(text))  # Hashtag
    special_characters = ['[UNAME]', '%', '$', '>', '/', ':', '(', '&', '_', '*', '^', ',', '#', '!', '~', ')', '=', '+', '-', '?', 'á—·á‘Œá–‡iá˜œ']
    for char in special_characters:
        text = text.replace(char, '')
    text = ''.join(char for char in text if unicodedata.category(char)[0] != 'C')  # Remove non-ASCII characters
    # Remove Emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # Remove Dupes
    text = re.sub(r'(.{1,})\1{2,}',r'\1\1', str(text))
    # Remove "\n" or ENTER
    text = text.replace('\n', ' ') 
    return text

In [23]:
import unicodedata
# Apply additional text cleaning to 'content' column
df['clean_content'] = df['content'].apply(additional_text_cleaning)

In [24]:
# Function to replace invalid characters
def replace_invalid_characters(text):
    mapping = {
        'ÄŸ': 'ğ',
        'Ã¼': 'ü',
        'Ä±': 'ı',
        'ÅŸ': 'ş',
        'Ã§': 'ç',
        'Ã¶': 'ö',
        'Ä°': 'İ',
        'Åž': 'Ş',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'Ã„Â±': 'ı',
        'ÅŸ': 'ş',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'ÄŸ': 'ğ',
        'Ãœ': 'Ü',
        'Ã¤Â±': 'ı',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ÄŸ': 'ğ',
        'Ã‡': 'Ç',
        'Ã–': 'Ö',
        'Ä°': 'İ',
        'ÅŸ': 'ş',
        'Ä±': 'ı',
        'ğ': 'ğ',
        'ü': 'ü',
        'ı': 'ı',
        'ş': 'ş',
        'ç': 'ç',
        'ö': 'ö',
        'İ': 'İ',
        'Ş': 'Ş',
        'Ç': 'Ç',
        'Ö': 'Ö',
        'Ü': 'Ü'
    }
    for char, repl_char in mapping.items():
        text = text.replace(char, repl_char)
    return text



In [27]:
# Apply additional text cleaning to 'clean_content' column
df['clean_content'] = df['clean_content'].apply(replace_invalid_characters)

# Display first few rows after replacing invalid characters
print(df.head(1000))

                                           raw_content  \
0    Sorry walau orang desa ga ga bulluk. ga dekil ...   
1    Adzan buat panggilan kampanye semoga aja nggak...   
2    gw ga sampe bundir sih preventif biar ga stres...   
3                         Cupu https://t.co/tP58k5u17Z   
4    pengen banget sibuk ngurusin keperluan ospek, ...   
..                                                 ...   
995  punten, sabar ya angkatan 57 terutama SV tahun...   
996  @Komjen_Langit Makna sempurna itu luas - janga...   
997                       LO GA USAH SOK GANTENG ANJIR   
998  @fmuchtaar Itu org lucu bet, analisis gw sedik...   
999  @threadshopee @tanyakanrl Indirect kiss y anta...   

                                               content  \
0    Sorry walau orang desa ga ga bulluk. ga dekil ...   
1    Adzan buat panggilan kampanye semoga aja nggak...   
2    gw ga sampe bundir sih preventif biar ga stres...   
3                                                 Cupu   
4    pengen b

# Menghapus duplikasi data dan data yang kosong

In [53]:
# Drop Duplikasi Data
df['preprocess_text'] = df['preprocess_text'].apply(filteringText)
df.drop_duplicates(subset='clean_text', inplace=True)

In [30]:
# Export cleaned DataFrame to CSV
df.to_csv("/kaggle/working/cleaned_cyberbully.csv", index=False)

# Normaalisasi Data

In [35]:
kamus = pd.read_csv('/kaggle/input/normalisasi2/normalisasi.csv')

In [36]:
kamus['original'] = kamus['original'].astype(str)
dictionary = kamus.set_index('original').to_dict()['normalisasi']

In [37]:
# Create a copy of the dataframe for normalization and labeling
xx = df.copy()
# Ensure all values in 'content' are strings and handle NaN values
xx['content'] = xx['content'].astype(str).fillna('')

In [38]:
# Ensure all values in 'content' are strings and handle NaN values
xx['content'] = xx['content'].astype(str).fillna('')

In [34]:
# Function to filter out non-Indonesian words
def filteringText(text):
    indonesian_stopwords = set(stopwords.words('indonesian'))
    words = text.split()
    filtered_words = [word for word in words if word not in indonesian_stopwords]
    return ' '.join(filtered_words)

In [40]:
from nltk.corpus import stopwords
nltk.download('stopwords')
# Apply text preprocessing, remove repeated words, and normalization
xx['cleaned_content'] = xx['content'].apply(text_preprocessing).apply(remove_repeated_words)
df_result = replace_words_in_sentence(xx, 'cleaned_content', dictionary)
df_result['cleaned_content'] = df_result['cleaned_content'].apply(filteringText)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
print(df_result['cleaned_content'].head())

0    sorry orang desa bulluk dekil kaya tetangga ho...
1    adzan panggilan kampanye semoga bikin konflik ...
2    sampe bundir sih preventif biar stress ckup bu...
3                                                 cupu
4    pengen banget sibuk ngurusin keperluan ospek k...
Name: cleaned_content, dtype: object


In [15]:
# Save processed text into a new column
df['preprocessed_text'] = df_result['cleaned_content']


# Labeling Data

In [112]:
# Read positive lexicon
lexicon_positive = dict()
with open('/kaggle/input/lexiconkamus2/positive.tsv', 'r', encoding='utf-8', newline='') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    next(reader)  # Skip header
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

In [113]:
# Read negative lexicon
lexicon_negative = dict()
with open('/kaggle/input/lexiconkamus2/negative.tsv', 'r', encoding='utf-8', newline='') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    next(reader)  # Skip header
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

In [114]:
df.head(1000)

,content,clean_text,preprocess_text
0,Sorry walau orang desa ga ga bulluk. ga dekil ...,sorry walau orang desa ga ga bulluk ga dekil k...,"[sorry, orang, desa, ga, ga, bulluk, ga, dekil..."
1,Adzan buat panggilan kampanye semoga aja nggak...,adzan buat panggilan kampanye semoga aja nggak...,"[adzan, panggil, kampanye, moga, aja, nggak, b..."
2,gw ga sampe bundir sih preventif biar ga stres...,gw ga sampe bundir sih preventif biar ga stres...,"[gw, ga, sampe, bundir, sih, preventif, biar, ..."
3,Cupu,cupu,[cupu]
4,"pengen banget sibuk ngurusin keperluan ospek, ...",pengen banget sibuk ngurusin keperluan ospek k...,"[ken, banget, sibuk, ngurusin, perlu, ospek, p..."
...,...,...,...
995,"punten, sabar ya angkatan 57 terutama SV tahun...",punten sabar ya angkatan terutama sv tahun osp...,"[punten, sabar, ya, angkat, sv, ospek, online,..."
996,Makna sempurna itu luas - jangan terjebak deng...,makna sempurna itu luas jangan terjebak dengan...,"[makna, sempurna, luas, jebak, sempurna, km, b..."
997,LO GA USAH SOK GANTENG ANJIR,lo ga usah sok ganteng anjir,"[lo, ga, sok, ganteng, anjir]"
998,"Itu org lucu bet, analisis gw sedikit islamoph...",itu org lucu bet analisis gw sedikit islamopho...,"[org, lucu, bet, analisis, gw, islamophobia, t..."


In [60]:
# Export cleaned DataFrame to CSV
df.to_csv("/kaggle/working/cleaned_cyberbully.csv", index=False)

In [115]:
# Function to analyze sentiment
def analysisSentiment(words):
    score = 0

    # Calculate score based on positive and negative lexicons
    for word in words:
        if word in lexicon_positive:
            score += lexicon_positive[word]
        if word in lexicon_negative:
            score -= lexicon_negative[word]

    # Determine polarity based on score
    if score > 5:
        polarity = 'Bullying'
    else:
        polarity = 'Not Bullying'

    return score, polarity

In [116]:
# Apply sentiment analysis
results = df['preprocess_text'].apply(analysisSentiment)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]

In [117]:
# Display polarity value counts
print(df['polarity'].value_counts())

polarity
Bullying        3013
Not Bullying     579
Name: count, dtype: int64


In [118]:
# Display first few rows of the dataframe with new sentiment columns
print(df.head(20))

                                              content  \
0   Sorry walau orang desa ga ga bulluk. ga dekil ...   
1   Adzan buat panggilan kampanye semoga aja nggak...   
2   gw ga sampe bundir sih preventif biar ga stres...   
3                                                Cupu   
4   pengen banget sibuk ngurusin keperluan ospek, ...   
5   Aku benci banget sama satu budheku. Setiap kal...   
6   Biar gaada pembeda, kalian itu di ospek sama s...   
7                   Itu pinokio, boneka goblok dungu,   
8   malam semuanyaa!! ‍️ boleh tolong ospekin setu...   
9   新しい : ospEk timE! ★ hello, kindly ospekin anak...   
10               Ngelatih mental kok ospek, nikah lah   
11  Help repost/retweet please? Hai! Boleh bantu o...   
12             help ospek &amp; sarannnya teman-teman   
13  But mamaku sampe detik ini masih percaya canti...   
14  NAJISSSSS!!! Badan gw cakep juga bukan buat la...   
15                               Dulu jelek dan kurus   
16  Ternyata, tidak ada hal yan

In [126]:
# Cek dataset jika ada yang duplikat dan jika ada yang kosong dihapus
hashable_columns = [col for col in df.columns if df[col].dtype != 'object' or df[col].apply(lambda x: isinstance(x, list)).sum() == 0]
df_no_duplicates = df.drop_duplicates(subset=hashable_columns)

In [127]:
# Hapus baris yang memiliki nilai kosong
df_cleaned = df_no_duplicates.dropna()

In [129]:
# Display polarity value counts
print(df['polarity'].value_counts())

polarity
Bullying        3013
Not Bullying     579
Name: count, dtype: int64


In [128]:
# Export cleaned DataFrame to CSV
df.to_csv("/kaggle/working/labeling_cyberbully_steming2.csv", index=False)